In [1]:
from __future__ import print_function
from PYME.recipes import modules
from PYME.LMVis import pipeline
data = pipeline.Pipeline("c:/users/work/#WORK_NOTEBOOK/CS_Notebook+Github_projects/GitHub/python-group-projects/data/31_5_series_D.h5r")

Trying to load 3rd party recipe module PYMEcs.recipes.processing
Loaded 3rd party recipe module PYMEcs.recipes.processing
Trying to load 3rd party recipe module PYMEnf.recipes.pymenf
Loaded 3rd party recipe module PYMEnf.recipes.pymenf
Trying to load 3rd party recipe module PYMEcs.recipes.output
Loaded 3rd party recipe module PYMEcs.recipes.output
Trying to load 3rd party recipe module PYMEcs.recipes.localisations
Loaded 3rd party recipe module PYMEcs.recipes.localisations
Trying to load 3rd party recipe module PYMEcs.recipes.base
Loaded 3rd party recipe module PYMEcs.recipes.base


In [2]:
# load our saved recipe
data.recipe.update_from_yaml('example1.yaml')
print("Current data source: %s" % data.selectedDataSourceKey) # what is the currently selected data source
print("Available data sources: %s" % ', '.join(data.dataSources.keys())) # look at all the data sources we could select
data.selectDataSource('cytosolC')
#print("Number of events in data source cytosolC is %d" % data.x.size)

Current data source: filtered_localizations
Available data sources: cytosolC, coalesced, with_clumps, filtered_localizations, Localizations, clumps, colour_mapped


In [ ]:
from PYME.recipes import localisations
tb = localisations.TimeBlocks(input='cytosolC',output='tbCytosolC',
                                 block_size=100)
data.recipe.add_module(tb)
dm = localisations.DensityMapping(
        inputLocalizations='tbCytosolC',
        jitterVariable="error_x",
        outputImage="FRCCytosolC",
        pixelSize=5.0,
        renderingModule='Gaussian',
        colours=['block0','block1'])
data.recipe.add_module(dm)
data.recipe.execute()

## This approach works for the above modules but I couldn't get it to work for a filter table.
### Annoyingly I think it is something simple I am missing.

In [37]:
from PYME.recipes import tablefilters
                                                                              ## Missing filters
changesize = tablefilters.FilterTable(inputName ='cytosolC',output='changedsize', x=[250, 750])
data.recipe.add_module(changesize)
data.recipe.execute()



AttributeError: 'dict' object has no attribute 'inputs'

In [34]:
from PYME.recipes import tablefilters
x = [i for i in range(250,750)]
#print (x)
changesize = tablefilters.FilterTable(outputName= 'changedsize', inputName= 'cytosolC', filters={'x': x})

data.recipe.add_module(changesize)
data.recipe.execute()

AttributeError: 'dict' object has no attribute 'inputs'

In [5]:
print("Current data source: %s" % data.selectedDataSourceKey) # what is the currently selected data source
print("Available data sources: %s" % ', '.join(data.dataSources.keys())) # look at all the data sources we could select
data.selectDataSource('changedsize')


Current data source: cytosolC
Available data sources: cytosolC, coalesced, with_clumps, filtered_localizations, filtered, Localizations, clumps, colour_mapped


KeyError: 'Data Source "changedsize" not found'

## Instead I had to do this, creating a new .yaml with my addition. 

In [ ]:
import yaml
with open('example1.yaml') as f:
    Resized = yaml.load(f, Loader=yaml.FullLoader)
    
import sys
import ruamel.yaml
record_to_add = dict({'tablefilters.FilterTable': {'outputName': 'Resized', 'inputName': 'cytosolC', 'filters': {'x': [250, 750],'y': [250, 750]}}})

yaml = ruamel.yaml.YAML()
yaml.explicit_start = True
Resized.append(record_to_add)
yaml.dump(Resized, sys.stdout)
with open(r'example_2.yaml', 'w') as file:
    Resized2 = yaml.dump(Resized, file)